# Report

## Dataset
two moon dataset with noise 0.1, 0.3, 0.5, 0.7, 0.9
- x : 2 x ?  
- y : 1 x ?

## Logistic Regression
| Logistic Regression | 0.1  | 0.3  | 0.5  | 0.7   | 0.9   |
|---------------------|------|------|------|-------|-------|
| accuracy            | 0.84 | 0.85 | 0.85 | 0.755 | 0.785 |

You can see that accuracy generally decreases as noise gets bigger, which is quite natural, because noise represents inconsistent, unseperable, faulty data.

## One Hidden Layer NN
| One Hidden Layer | 0.1   | 0.3   | 0.5   | 0.7   | 0.9   |
|------------------|-------|-------|-------|-------|-------|
| 1                | 0.84  | 0.85  | 0.855 | 0.76  | 0.78  |
| 3                | 1.0   | 0.895 | 0.84  | 0.755 | 0.795 |
| 5                | 0.84  | 0.84  | 0.865 | 0.765 | 0.785 |
| 7                | 0.845 | 0.895 | 0.865 | 0.725 | 0.78  |
| 9                | 0.845 | 0.895 | 0.865 | 0.73  | 0.078 |
| Optimal          | 3     | 3     | 5     | 5     | 3     |

I chose the optimal layer with the standard of
- having the highest accuracy in that dataset
- if accuracy score is the same, choose the one that has less hidden units (simpler)

You can see that accuracy generally decreases as noise gets bigger.  

You can also see that 3~5 hidden units have the best accuracy, and even if you have more(or less) hidden units than that, it doesn't help you in raising accuracy, rather it lowers the accuracy.

## Two Hidden Layer NN
| Two Hidden Layer | 0.1  | 0.3   | 0.5   | 0.7   | 0.9   |
|------------------|------|-------|-------|-------|-------|
| 1x1              | 0.84 | 0.84  | 0.855 | 0.76  | 0.775 |
| 1x3              | 0.84 | 0.84  | 0.855 | 0.76  | 0.775 |
| 1x5              | 0.84 | 0.84  | 0.855 | 0.765 | 0.775 |
| 1x7              | 0.84 | 0.84  | 0.855 | 0.76  | 0.775 |
| 1x9              | 0.84 | 0.84  | 0.855 | 0.76  | 0.775 |
| 3x1              | 1.0  | 0.895 | 0.865 | 0.735 | 0.775 |
| 3x3              | 1.0  | 0.835 | 0.855 | 0.735 | 0.775 |
| 3x5              | 1.0  | 0.895 | 0.86  | 0.765 | 0.78  |
| 3x7              | 1.0  | 0.9   | 0.86  | 0.755 | 0.775 |
| 3x9              | 1.0  | 0.895 | 0.86  | 0.765 | 0.775 |
| 5x1              | 1.0  | 0.835 | 0.85  | 0.735 | 0.78  |
| 5x3              | 1.0  | 0.895 | 0.86  | 0.755 | 0.775 |
| 5x5              | 1.0  | 0.895 | 0.865 | 0.725 | 0.775 |
| 5x7              | 1.0  | 0.84  | 0.855 | 0.755 | 0.775 |
| 5x9              | 1.0  | 0.895 | 0.86  | 0.76  | 0.785 |
| 7x1              | 1.0  | 0.895 | 0.86  | 0.735 | 0.465 |
| 7x3              | 1.0  | 0.89  | 0.855 | 0.735 | 0.785 |
| 7x5              | 1.0  | 0.89  | 0.87  | 0.735 | 0.785 |
| 7x7              | 1.0  | 0.89  | 0.86  | 0.735 | 0.78  |
| 7x9              | 1.0  | 0.895 | 0.86  | 0.735 | 0.78  |
| 9x1              | 1.0  | 0.49  | 0.86  | 0.535 | 0.455 |
| 9x3              | 1.0  | 0.895 | 0.865 | 0.715 | 0.785 |
| 9x5              | 1.0  | 0.895 | 0.845 | 0.755 | 0.78  |
| 9x7              | 1.0  | 0.89  | 0.85  | 0.735 | 0.78  |
| 9x9              | 1.0  | 0.895 | 0.87  | 0.755 | 0.795 |
| Optimal          | 3x1  | 3x7   | 7x5   | 1x5   | 9x9   |

I chose the optimal layer with the standard of
- having the highest accuracy in that dataset
- if accuracy score is the same, choose the one that has less hidden units (simpler)

You can see that accuracy generally decreases as noise gets bigger.  

When there are less noise, having more layers can generally improve accuracy.  
When there are too much noise, having more layers won't help that much, because the train data doesn't represent the test data so much.  

The data has more relation to the number of hidden layers in the first layer, and less relation to the second layer, as you can see clearly in noise 0.1.

## Overall


I stopped training when the cost of validation set starts to increase while training, to prevent overfitting.  
Maybe this lead to a learning failure in some complicated networks, since I didn't give them enough time for the cost function to constantly decrease in the early iterations. You could see some accuracy scores stopping on near 0.5, which is caused by a break in the loop for gaining worse scores on validation set. To prevent this, I think I should give more iterations until I evaluate the validation score in complex models.

You could see that there are times when validation cost is higher, but test accuracy is also higher.  
This could happen because we use a different validation set and test set.  

My gradient calculation is all based on the course pdf. (Which helped A LOT thank you)  
The accuracy calculation is based on how many inferences the model got it correct in the test dataset, which the model hasn't seen in training.

When there is too much noise, the train data doesn't represent the true data, which means complicated classifiers won't have much of a meaning, since it would only try to learn more of the train data.  
Even when there is less noise, if you see the chart of one hidden layer noise 0.1, complex models(5,7,9) did not beat 3 hidden units, which means they tried to overfit before they learned the general tendency of the data, leading to a validation score loss and thus a termination on learning.

If you see the two hidden layer case, you could see some correlation in the number of first layers. I think this is due to gradient vanishing, so the first layer gets less different than the second layer, resulting to similar output with the same number of first layers.

I learned that complicated networks are hard to learn, and it is not necessarily better in accuracy too.  
I think the optimal network is decided by how the data looks like, and I think the two moon dataset can be naturally classified in only 3 hidden units, so more than that could cause only overfitting.

Full output can be seen below.

In [ ]:
import numpy as np # only required to use numpy

In [2]:
class LogisticRegression:
    def __init__(self):
        '''
            matrix shape (row x column)
                x : 2 x ?
                w : 2 x 1
                b : 1 x 1
                wTx : 1 x ?
                y : 1 x ?
        '''
        self.w = np.random.rand(2,1) # weight initialization
        self.b = 0 # bias initialization
        
    def sigmoid(self, z):
        '''
            Activation function of logistic regression
            input :
                z : 1 x ?
            output :
                prediction(y_hat) : 1 x ?
        '''
        return 1 / (1 + np.exp(-z))

    def predict(self, x, classify=False):
        '''
            Predict using logistic regression
            input : 
                data(x) : 2 x ?
            output : 
                prediction(y_hat) : 1 x ?
        '''
        z = self.w.T @ x + self.b # z : 1 x ?
        a = self.sigmoid(z) # a : 1 x ?
        
        if classify == True:
            classifier = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
            return classifier(a)
        else:
            return a


    def cost(self, y_hat, y):
        '''
            Cost function of logistic regression
            input : 
                prediction(y_hat) : 1 x ?
                label(y) : 1 x ?
            output : 
                cost function value : 1 x 1
        '''
        assert len(y_hat) == len(y)

        m = y.shape[1]

        epsilon = 0.000000001 # prevent -inf in log operation

        cost = (-1 / m) * np.sum(y * np.log(y_hat + epsilon) + (1-y) * np.log(1 - y_hat + epsilon))

        return cost

    def gradient_descent(self, x, y, lr):
        y_hat = self.predict(x)
        
        m = y.shape[1]
        
        dz = y_hat - y # dz : 1 x ?
        dw = (1 / m) * (x @ dz.T) # x : 2 x ?, w : 2 x 1
        db = (1 / m) * np.sum(dz) # b : 1 x 1
        
        self.w -= lr * dw
        self.b -= lr * db

    def train(self, train_x, train_y, val_x, val_y, lr, iters):
        for i in range(iters):
            if i % 10000 == 0:
                prev_w = self.w
                prev_b = self.b
                prev_cost = self.cost(self.predict(val_x, classify=False), val_y)
            
            self.gradient_descent(train_x, train_y, lr) # update w,b

            # stop training if cost has increased in validation set : prevent overfitting
            if i % 10000 == 9999:
                current_cost = self.cost(self.predict(val_x, classify=False), val_y)
                if prev_cost < current_cost: 
                    self.w = prev_w
                    self.b = prev_b
                    current_cost = prev_cost
                    break
            
        print("Iteration:", i ,"Cost:", current_cost)
                
    def accuracy(self, test_x, test_y):
        y_hat = self.predict(test_x, classify=True)
        return (np.sum(y_hat == test_y) / test_y.shape[1])

In [3]:
class OneHiddenLayer:
    def __init__(self, hidden):
        '''
            matrix shape (row x column)
                x : 2 x ?
                
                w1 : [1,3,5,7,9] x 2
                b1 : [1,3,5,7,9] x 1
                a1 : [1,3,5,7,9] x ?
                
                w2 : 1 x [1,3,5,7,9]
                b2 : 1 x 1
                a2(y_hat) : 1 x ?
        '''
        # layer 1 (hidden layer)
        self.w1 = np.random.rand(hidden, 2) # weight initialization
        self.b1 = np.zeros((hidden, 1)) # bias initialization
        # final layer (goes to y)
        self.w2 = np.random.rand(1, hidden)
        self.b2 = 0
        
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def predict(self, x, classify=False):
        
        z1 = self.w1 @ x + self.b1 # z1 : [1,3,5,7,9] x ?
        a1 = self.sigmoid(z1) # a1 : [1,3,5,7,9] x ?
        z2 = self.w2 @ a1 + self.b2 # z2 : 1 x ?
        a2 = self.sigmoid(z2) # a2 : 1 x ?
        
        if classify == True:
            classifier = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
            return classifier(a2)
        else:
            return a2


    def cost(self, y_hat, y):
        assert len(y_hat) == len(y)

        m = y.shape[1]

        epsilon = 0.000000001 # prevent -inf in log operation

        cost = (-1 / m) * np.sum(y * np.log(y_hat + epsilon) + (1-y) * np.log(1 - y_hat + epsilon))

        return cost

    def gradient_descent(self, x, y, lr):
        
        z1 = self.w1 @ x + self.b1 # z1 : [1,3,5,7,9] x ?
        a1 = self.sigmoid(z1) # a1 : [1,3,5,7,9] x ?
        z2 = self.w2 @ a1 + self.b2 # z2 : 1 x ?
        a2 = self.sigmoid(z2) # a2 : 1 x ?
        
        m = y.shape[1]
        
        dz2 = a2 - y # dz2 : 1 x ?
        dw2 = (1 / m) * (dz2 @ a1.T) # dw2 : 1 x [1,3,5,7,9]
        db2 = (1 / m) * np.sum(dz2, axis=1, keepdims=True) # db2 : 1 x ? -> 1 x 1
        dz1 = (self.w2.T @ dz2) * (self.sigmoid_derivative(z1)) # dz1 : [1,3,5,7,9] x ? # element wise matrix multiplication
        dw1 = (1 / m) * (dz1 @ x.T) # [1,3,5,7,9] x 2
        db1 = (1 / m) * np.sum(dz1, axis=1, keepdims=True) # db1 : [1,3,5,7,9] x ? -> [1,3,5,7,9] x 1
        
        self.w2 -= lr * dw2
        self.b2 -= lr * db2
        self.w1 -= lr * dw1
        self.b1 -= lr * db1
        
        

    def train(self, train_x, train_y, val_x, val_y, lr, iters):
        for i in range(iters):
            if i % 10000 == 0:
                prev_w1 = self.w1
                prev_b1 = self.b1
                prev_w2 = self.w2
                prev_b2 = self.b2
                prev_cost = self.cost(self.predict(val_x, classify=False), val_y)
            
            self.gradient_descent(train_x, train_y, lr) # update w,b
            
            # stop training if cost has increased in validation set : prevent overfitting
            if i % 10000 == 9999:
                current_cost = self.cost(self.predict(val_x, classify=False), val_y)
                if prev_cost < current_cost: 
                    self.w1 = prev_w1
                    self.b1 = prev_b1
                    self.w2 = prev_w2
                    self.b2 = prev_b2
                    current_cost = prev_cost
                    break
            
        print("Iteration:", i ,"Cost:", current_cost)
                
    def accuracy(self, test_x, test_y):
        y_hat = self.predict(test_x, classify=True)
        return (np.sum(y_hat == test_y) / test_y.shape[1])

In [4]:
class TwoHiddenLayer:
    def __init__(self, hidden1, hidden2):
        '''
            matrix shape (row x column)
                x : 2 x ?
                
                w1 : hidden1 x 2
                b1 : hidden1 x 1
                a1 : hidden1 x ?
                
                w2 : hidden2 x hidden1
                b2 : hidden2 x 1
                a2 : hidden2 x ?
                
                w3 : 1 x hidden2
                b3 : 1 x 1
                a3(y_hat) : 1 x ?
        '''
        # layer 1 (hidden1)
        self.w1 = np.random.rand(hidden1, 2) # weight initialization
        self.b1 = np.zeros((hidden1, 1)) # bias initialization
        # layer 2 (hidden2)
        self.w2 = np.random.rand(hidden2, hidden1)
        self.b2 = np.zeros((hidden2, 1))
        # final layer (goes to y)
        self.w3 = np.random.rand(1, hidden2)
        self.b3 = 0
        
        
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derivative(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))

    def predict(self, x, classify=False):
        
        z1 = self.w1 @ x + self.b1 # z1 : hidden1 x ?
        a1 = self.sigmoid(z1) # a1 : hidden1 x ?
        z2 = self.w2 @ a1 + self.b2 # z2 : hidden2 x ?
        a2 = self.sigmoid(z2) # a2 : hidden2 x ?
        z3 = self.w3 @ a2 + self.b3 # z3 : 1 x ?
        a3 = self.sigmoid(z3) # a3 : 1 x ?
        
        if classify == True:
            classifier = np.vectorize(lambda x: 1 if x >= 0.5 else 0)
            return classifier(a3)
        else:
            return a3


    def cost(self, y_hat, y):
        assert len(y_hat) == len(y)

        m = y.shape[1]

        epsilon = 0.000000001 # prevent -inf in log operation

        cost = (-1 / m) * np.sum(y * np.log(y_hat + epsilon) + (1-y) * np.log(1 - y_hat + epsilon))

        return cost

    def gradient_descent(self, x, y, lr):
        
        z1 = self.w1 @ x + self.b1 # z1 : hidden1 x ?
        a1 = self.sigmoid(z1) # a1 : hidden1 x ?
        z2 = self.w2 @ a1 + self.b2 # z2 : hidden2 x ?
        a2 = self.sigmoid(z2) # a2 : hidden2 x ?
        z3 = self.w3 @ a2 + self.b3 # z3 : 1 x ?
        a3 = self.sigmoid(z3) # a3 : 1 x ?
        
        m = y.shape[1]
        
        dz3 = a3 - y
        dw3 = (1 / m) * (dz3 @ a2.T)
        db3 = (1 / m) * np.sum(dz3, axis=1, keepdims=True)
        
        dz2 = (self.w3.T @ dz3) * (self.sigmoid_derivative(z2))
        dw2 = (1 / m) * (dz2 @ a1.T)
        db2 = (1 / m) * np.sum(dz2, axis=1, keepdims=True)
        
        dz1 = (self.w2.T @ dz2) * (self.sigmoid_derivative(z1))
        dw1 = (1 / m) * (dz1 @ x.T)
        db1 = (1 / m) * np.sum(dz1, axis=1, keepdims=True)
        
        self.w3 -= lr * dw3
        self.b3 -= lr * db3
        self.w2 -= lr * dw2
        self.b2 -= lr * db2
        self.w1 -= lr * dw1
        self.b1 -= lr * db1
        

    def train(self, train_x, train_y, val_x, val_y, lr, iters):
        for i in range(iters):
            if i % 10000 == 0:
                prev_w1 = self.w1
                prev_b1 = self.b1
                prev_w2 = self.w2
                prev_b2 = self.b2
                prev_w3 = self.w3
                prev_b3 = self.b3
                prev_cost = self.cost(self.predict(val_x, classify=False), val_y)
            
            self.gradient_descent(train_x, train_y, lr) # update w,b
            
            # stop training if cost has increased in validation set : prevent overfitting
            if i % 10000 == 9999:
                current_cost = self.cost(self.predict(val_x, classify=False), val_y)
                if prev_cost < current_cost: 
                    self.w1 = prev_w1
                    self.b1 = prev_b1
                    self.w2 = prev_w2
                    self.b2 = prev_b2
                    self.w3 = prev_w3
                    self.b3 = prev_b3
                    current_cost = prev_cost
                    break
            
        print("Iteration:", i ,"Cost:", current_cost)
                
    def accuracy(self, test_x, test_y):
        y_hat = self.predict(test_x, classify=True)
        return (np.sum(y_hat == test_y) / test_y.shape[1])

In [5]:
# input data

noises = [0.1, 0.3, 0.5, 0.7, 0.9]
data = [{"train" : None, "val": None, "test": None} for _ in range(len(noises))]

for i in range(len(noises)):
    train_x = []
    train_y = []
    train_file = open("two_moon_{}/train.txt".format(noises[i]), "r")
    for line in train_file.readlines():
        l = line.rstrip().split()
        train_x.append([float(l[0]), float(l[1])])
        train_y.append([int(l[2])])
    train_file.close()
    data[i]["train"] = (np.array(train_x).T, np.array(train_y).T)

    val_x = []
    val_y = []
    val_file = open("two_moon_{}/val.txt".format(noises[i]), "r")
    for line in val_file.readlines():
        l = line.rstrip().split()
        val_x.append([float(l[0]), float(l[1])])
        val_y.append([int(l[2])])
    val_file.close()
    data[i]["val"] = (np.array(val_x).T, np.array(val_y).T)

    test_x = []
    test_y = []
    test_file = open("two_moon_{}/test.txt".format(noises[i]), "r")
    for line in test_file.readlines():
        l = line.rstrip().split()
        test_x.append([float(l[0]), float(l[1])])
        test_y.append([int(l[2])])
    test_file.close()
    data[i]["test"] = (np.array(test_x).T, np.array(test_y).T)

In [6]:
# start training!

for i in range(len(noises)):
    train_x, train_y = data[i]["train"]
    val_x, val_y = data[i]["val"]
    test_x, test_y = data[i]["test"]
    
    print("Noise : ", noises[i])
    print("################################")
    
    print("Logistic Regression")
    prob_1 = LogisticRegression()
    prob_1.train(train_x, train_y, val_x, val_y, lr=0.01, iters=1000000)
    print("Accuracy:", prob_1.accuracy(test_x, test_y))
    print("################################")
    
    for n in range(1,10,2):    
        print("One Hidden Layer : {} Layers".format(n))
        prob_2 = OneHiddenLayer(hidden=n)
        prob_2.train(train_x, train_y, val_x, val_y, lr=0.01, iters=1000000)
        print("Accuracy:", prob_2.accuracy(test_x, test_y))
    print("################################")
    
    for n1 in range(1,10,2):
        for n2 in range(1,10,2):
            print("Two Hidden Layers : {} x {} Layers".format(n1, n2))
            prob_3 = TwoHiddenLayer(hidden1=n1, hidden2=n2)
            prob_3.train(train_x, train_y, val_x, val_y, lr=0.01, iters=1000000)
            print("Accuracy:", prob_3.accuracy(test_x, test_y))
    print("################################")

Noise :  0.1
################################
Logistic Regression
Iteration: 39999 Cost: 0.2736342444626414
Accuracy: 0.84
################################
One Hidden Layer : 1 Layers
Iteration: 999999 Cost: 0.2785162473832272
Accuracy: 0.84
One Hidden Layer : 3 Layers
Iteration: 999999 Cost: 0.008456047660195986
Accuracy: 1.0
One Hidden Layer : 5 Layers
Iteration: 29999 Cost: 0.2820930675915389
Accuracy: 0.84
One Hidden Layer : 7 Layers
Iteration: 29999 Cost: 0.27977694881654624
Accuracy: 0.845
One Hidden Layer : 9 Layers
Iteration: 29999 Cost: 0.2791618555972905
Accuracy: 0.845
################################
Two Hidden Layers : 1 x 1 Layers
Iteration: 999999 Cost: 0.27864589496320924
Accuracy: 0.84
Two Hidden Layers : 1 x 3 Layers
Iteration: 999999 Cost: 0.2774905610710093
Accuracy: 0.84
Two Hidden Layers : 1 x 5 Layers
Iteration: 999999 Cost: 0.27737202105843706
Accuracy: 0.84
Two Hidden Layers : 1 x 7 Layers
Iteration: 999999 Cost: 0.2773306638742863
Accuracy: 0.84
Two Hidden Lay